# Preprocesamiento datos SECOP I

## 1. Importar librerías

Iniciar sesión de Spark

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("SECOP").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # para generar mejor formato de tablas
spark

Importar librerías

In [2]:
# Librerías básicas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import *

## 2. Lectura base de datos SECOP I

In [3]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType, LongType

# Definir estructura para lectura de base de datos SECOP I
schema_secop = StructType([ \
    StructField('UID', StringType(), True), \
    StructField('Anno Cargue SECOP', IntegerType(), True), \
    StructField('Anno Firma del Contrato', IntegerType(), True), \
    StructField('Nivel Entidad', StringType(), True), \
    StructField('Orden Entidad', StringType(), True), \
    StructField('Nombre de la Entidad', StringType(), True), \
    StructField('NIT de la Entidad', StringType(), True), \
    StructField('Código de la Entidad', StringType(), True), \
    StructField('ID Tipo de Proceso', StringType(), True), \
    StructField('Tipo de Proceso', StringType(), True), \
    StructField('Estado del Proceso', StringType(), True), \
    StructField('Causal de Otras Formas de Contratacion Directa', StringType(), True), \
    StructField('ID Regimen de Contratacion', StringType(), True), \
    StructField('Regimen de Contratacion', StringType(), True), \
    StructField('ID Objeto a Contratar', StringType(), True), \
    StructField('Objeto a Contratar', StringType(), True), \
    StructField('Detalle del Objeto a Contratar', StringType(), True), \
    StructField('Tipo de Contrato', StringType(), True), \
    StructField('Municipio Obtencion', StringType(), True), \
    StructField('Municipio Entrega', StringType(), True), \
    StructField('Municipios Ejecucion', StringType(), True), \
    StructField('Fecha de Cargue en el SECOP', TimestampType(), True), \
    StructField('Numero de Constancia', StringType(), True), \
    StructField('Numero de Proceso', StringType(), True), \
    StructField('Numero del Contrato', StringType(), True), \
    StructField('Cuantia Proceso', StringType(), True), \
    StructField('ID Grupo', StringType(), True), \
    StructField('Nombre Grupo', StringType(), True), \
    StructField('ID Familia', StringType(), True), \
    StructField('Nombre Familia', StringType(), True), \
    StructField('ID Clase', StringType(), True), \
    StructField('Nombre Clase', StringType(), True), \
    StructField('ID Ajudicacion', StringType(), True), \
    StructField('Tipo Identifi del Contratista', StringType(), True), \
    StructField('Identificacion del Contratista', StringType(), True), \
    StructField('Nom Raz Social Contratista', StringType(), True), \
    StructField('Dpto y Muni Contratista', StringType(), True), \
    StructField('Tipo Doc Representante Legal', StringType(), True), \
    StructField('Identific del Represen Legal', StringType(), True), \
    StructField('Nombre del Represen Legal', StringType(), True), \
    StructField('Fecha de Firma del Contrato', TimestampType(), True), \
    StructField('Fecha Ini Ejec Contrato', TimestampType(), True), \
    StructField('Plazo de Ejec del Contrato', IntegerType(), True), \
    StructField('Rango de Ejec del Contrato', StringType(), True), \
    StructField('Tiempo Adiciones en Dias', IntegerType(), True), \
    StructField('Tiempo Adiciones en Meses', IntegerType(), True), \
    StructField('Fecha Fin Ejec Contrato', TimestampType(), True), \
    StructField('Compromiso Presupuestal', StringType(), True), \
    StructField('Cuantia Contrato', StringType(), True), \
    StructField('Valor Total de Adiciones', StringType(), True), \
    StructField('Valor Contrato con Adiciones', StringType(), True), \
    StructField('Objeto del Contrato a la Firma', StringType(), True), \
    StructField('ID Origen de los Recursos', StringType(), True), \
    StructField('Origen de los Recursos', StringType(), True), \
    StructField('Codigo BPIN', StringType(), True), \
    StructField('Proponentes Seleccionados', StringType(), True), \
    StructField('Calificacion Definitiva', StringType(), True), \
    StructField('ID Sub Unidad Ejecutora', StringType(), True), \
    StructField('Nombre Sub Unidad Ejecutora', StringType(), True), \
    StructField('Ruta Proceso en SECOP I', StringType(), True), \
    StructField('Moneda', StringType(), True), \
    StructField('EsPostConflicto', StringType(), True), \
    StructField('Marcacion Adiciones', StringType(), True), \
    StructField('Posicion Rubro', StringType(), True), \
    StructField('Nombre Rubro', StringType(), True), \
    StructField('Valor Rubro', StringType(), True), \
    StructField('Sexo RepLegal Entidad', StringType(), True), \
    StructField('Pilar Acuerdo Paz', StringType(), True), \
    StructField('Punto Acuerdo Paz', StringType(), True), \
    StructField('Municipio Entidad', StringType(), True), \
    StructField('Departamento Entidad', StringType(), True), \
    StructField('Ultima Actualizacion', TimestampType(), True), \
    StructField('Fecha Liquidacion', TimestampType(), True)
  ])

In [4]:
# Cargar base de datos SECOP I en Spark DataFrame
df_secop_crudo = spark.read.csv("SECOP.csv", header=True, schema = schema_secop)
#df_secop_crudo.printSchema()

## 3. Limpieza de datos

### 3.1 Eliminar columnas innecesarias

In [5]:
# Eliminar columnas innecesarias
columnas_drop = ('Nombre de la Entidad', 'NIT de la Entidad', 'Código de la Entidad', 'ID Tipo de Proceso', \
                'Causal de Otras Formas de Contratacion Directa', 'ID Regimen de Contratacion', 'Regimen de Contratacion',\
                'ID Objeto a Contratar', 'Municipio Obtencion', 'Municipio Entrega', 'Municipios Ejecucion', \
                'Fecha de Cargue en el SECOP', 'Numero de Constancia', 'Numero de Proceso', 'Numero del Contrato', \
                'ID Grupo', 'ID Familia', 'ID Clase', 'ID Ajudicacion', 'Tipo Identifi del Contratista', \
                'Identificacion del Contratista', 'Nom Raz Social Contratista', 'Dpto y Muni Contratista', \
                'Tipo Doc Representante Legal', 'Identific del Represen Legal', 'Nombre del Represen Legal', \
                'Fecha de Firma del Contrato', 'Compromiso Presupuestal', 'Valor Total de Adiciones', \
                'ID Origen de los Recursos', 'Origen de los Recursos', 'Codigo BPIN', 'Proponentes Seleccionados', \
                'Calificacion Definitiva', 'ID Sub Unidad Ejecutora', 'Nombre Sub Unidad Ejecutora', \
                'Ruta Proceso en SECOP I', 'EsPostConflicto', 'Marcacion Adiciones', 'Posicion Rubro', \
                'Nombre Rubro', 'Valor Rubro', 'Sexo RepLegal Entidad', 'Pilar Acuerdo Paz', 'Punto Acuerdo Paz', \
                'Municipio Entidad', 'Ultima Actualizacion', 'Fecha Liquidacion')

df_secop = df_secop_crudo.drop(*columnas_drop)

### 3.2 Cambiar nombre de columnas de interés

In [6]:
# Cambiar nombre de columnas

df_secop = df_secop.withColumnRenamed('Anno Cargue SECOP', 'Anno_Cargue_SECOP'). \
                          withColumnRenamed('Anno Firma del Contrato', 'Anno_Firma_Contrato'). \
                          withColumnRenamed('Nivel Entidad', 'Nivel_Entidad'). \
                          withColumnRenamed('Orden Entidad', 'Orden_Entidad'). \
                          withColumnRenamed('Tipo de Proceso', 'Tipo_Proceso'). \
                          withColumnRenamed('Estado del Proceso', 'Estado_Proceso'). \
                          withColumnRenamed('Objeto a Contratar', 'Objeto_Contratar'). \
                          withColumnRenamed('Detalle del Objeto a Contratar', 'Detalle_Objeto_Contratar'). \
                          withColumnRenamed('Tipo de Contrato', 'Tipo_Contrato'). \
                          withColumnRenamed('Cuantia Proceso', 'Cuantia_Proceso'). \
                          withColumnRenamed('Nombre Grupo', 'Nombre_Grupo'). \
                          withColumnRenamed('Nombre Familia', 'Nombre_Familia'). \
                          withColumnRenamed('Nombre Clase', 'Nombre_Clase'). \
                          withColumnRenamed('Fecha Ini Ejec Contrato', 'Fecha_Ini_Ejec_Contrato'). \
                          withColumnRenamed('Plazo de Ejec del Contrato', 'Plazo_Ejec_Contrato'). \
                          withColumnRenamed('Rango de Ejec del Contrato', 'Rango_Ejec_Contrato'). \
                          withColumnRenamed('Tiempo Adiciones en Dias', 'Tiempo_Adiciones_Dias'). \
                          withColumnRenamed('Tiempo Adiciones en Meses', 'Tiempo_Adiciones_Meses'). \
                          withColumnRenamed('Fecha Fin Ejec Contrato', 'Fecha_Fin_Ejec_Contrato'). \
                          withColumnRenamed('Cuantia Contrato', 'Cuantia_Contrato'). \
                          withColumnRenamed('Valor Contrato con Adiciones', 'Valor_Contrato_Total'). \
                          withColumnRenamed('Objeto del Contrato a la Firma', 'Objeto_Contrato_firmado'). \
                          withColumnRenamed('Origen de los Recursos', 'Origen_Recursos'). \
                          withColumnRenamed('Departamento Entidad', 'Departamento_Entidad')

### 3.3 Gestión de datos nulos

#### Ajustar formato de columnas de interés

In [7]:
# Imputar con cero columnas que tienen números enteros
cols_integer = ["Anno_Cargue_SECOP", "Anno_Firma_Contrato", "Cuantia_Proceso", "Plazo_Ejec_Contrato", \
               "Tiempo_Adiciones_Dias", "Tiempo_Adiciones_Meses", "Cuantia_Contrato", "Valor_Contrato_Total"]

df_secop = df_secop.fillna(0, subset = cols_integer)

In [8]:
# Imputar fechas con valores nulos
cols_date = ['Fecha_Ini_Ejec_Contrato', 'Fecha_Fin_Ejec_Contrato']

df_secop = df_secop.fillna('1900-01-01 00:00:00', subset = cols_date)
#from datetime import datetime, date
#fecha_nulo = datetime(1900, 1, 1, 0, 0)
#df_secop = df_secop.fillna(str(fecha_nulo), subset = cols_date)

In [9]:
# Imputar valores de texto nulos
cols_str = ['UID', 'Nivel_Entidad', 'Orden_Entidad', 'Tipo_Proceso', 'Estado_Proceso', \
            'Objeto_Contratar', 'Detalle_Objeto_Contratar', 'Tipo_Contrato',  \
            'Nombre_Grupo', 'Nombre_Familia', 'Nombre_Clase', 'Rango_Ejec_Contrato', \
            'Objeto_Contrato_firmado', 'Moneda', 'Departamento_Entidad']

df_secop = df_secop.fillna('No Definido', subset = cols_str)

In [10]:
## Procedimiento para cambiar formato de campos con valores de dinero (1)
# Quitar comas
from pyspark.sql.functions import udf

# Quitar comas de campos con valores de dinero
quitar_coma = udf(lambda x: x.replace(',',''))

df_secop = df_secop.withColumn('Cuantia_Proceso_Int',quitar_coma('Cuantia_Proceso')). \
                    withColumn('Cuantia_Contrato_Int',quitar_coma('Cuantia_Contrato')). \
                    withColumn('Valor_Contrato_Total_Int',quitar_coma('Valor_Contrato_Total'))

In [11]:
## Procedimiento para cambiar formato de campos con valores de dinero (2)
# Quitar números decimales (están precedidos de un punto "." )
df_secop = df_secop.withColumn("Cuantia_Proceso_Int", F.regexp_extract('Cuantia_Proceso_Int', '[0-9]+', 0)). \
                    withColumn("Cuantia_Contrato_Int", F.regexp_extract('Cuantia_Contrato_Int', '[0-9]+', 0)). \
                    withColumn("Valor_Contrato_Total_Int", F.regexp_extract('Valor_Contrato_Total_Int', '[0-9]+', 0))

In [12]:
## Procedimiento para cambiar formato de campos con valores de dinero (3)
# Cambiar formato de str a long
df_secop = df_secop.withColumn("Cuantia_Proceso_Int",df_secop.Cuantia_Proceso_Int.cast(LongType())). \
                    withColumn("Cuantia_Contrato_Int",df_secop.Cuantia_Contrato_Int.cast(LongType())). \
                    withColumn("Valor_Contrato_Total_Int",df_secop.Valor_Contrato_Total_Int.cast(LongType()))

# Eliminar columnas con datos originales en formato texto
columnas_drop = ('Cuantia_Contrato', 'Valor_Contrato_Total', 'Cuantia_Proceso')
df_secop = df_secop.drop(*columnas_drop)

### 3.4 Agregar nueva información a los datos

#### Agregar valores Año

In [13]:
# Agregar columnas indicando el año de inicio y el año de fin del contrato y dejar en formato int
df_secop = df_secop.withColumn("Anno_Inicio_Contrato",year('Fecha_Ini_Ejec_Contrato')). \
                    withColumn("Anno_Fin_Contrato",year('Fecha_Fin_Ejec_Contrato'))

columnas_drop = ('Fecha_Ini_Ejec_Contrato', 'Fecha_Fin_Ejec_Contrato')
df_secop = df_secop.drop(*columnas_drop)

df_secop = df_secop.fillna(0, subset = ['Anno_Inicio_Contrato', 'Anno_Fin_Contrato'])

df_secop = df_secop.withColumn("Anno_Inicio_Contrato",df_secop['Anno_Inicio_Contrato'].cast(IntegerType())). \
                    withColumn("Anno_Fin_Contrato",df_secop['Anno_Fin_Contrato'].cast(IntegerType()))

#### Modificar Departamento (nombre DANE) y agregar Región DANE

In [14]:
# Definir estructura para lectura de base de datos SECOP I
schema_dpto_reg = StructType([ \
    StructField('Dpto_SECOP', StringType(), True), \
    StructField('Dpto_DANE', StringType(), True), \
    StructField('Region_DANE', StringType(), True)
  ])

# Cargar información DANE y equivalencia del Departamento con SECOP
df_dpto_reg = spark.read.option("delimiter", ";").option("header", True).csv("Regiones_Departamentos.csv")
df_dpto_reg.printSchema()

# Crear diccionarios para cambiar departamento (según descripción DANE) y agregar Región DANE
df_dpto_reg_pd = df_dpto_reg.toPandas()
df_dpto_reg_pd.set_index('Dpto_SECOP', inplace = True)
dict_dpto = df_dpto_reg_pd['Dpto_DANE'].to_dict()  # Diccionario Dpto Secop -> DANE
dict_reg = df_dpto_reg_pd['Region_DANE'].to_dict()  # Diccionario Dpto DANE -> Región DANE

root
 |-- Dpto_SECOP: string (nullable = true)
 |-- Dpto_DANE: string (nullable = true)
 |-- Region_DANE: string (nullable = true)



In [15]:
# Crear función para traducir según diccionario
def traductor(dictionary):
    return udf(lambda col: dictionary.get(col), StringType())

In [16]:
# Crear columna con la región correspondiente según clasificación del DANE
df_secop = df_secop.withColumn("Region", traductor(dict_reg)("Departamento_Entidad"))

In [17]:
# Modificar columna Departamento_Entidad con el nombre del departamenteo según el DANE
df_secop = df_secop.withColumn("Departamento_Entidad_DANE", traductor(dict_dpto)("Departamento_Entidad"))

In [18]:
# Eliminar columnas original Departamento Entidad y actualizar nombre
#columnas_drop = ('Departamento_Entidad')
df_secop = df_secop.drop(*columnas_drop)

### 3.5 Eliminar filas de datos no requeridos

In [19]:
# Dejar Estado del Proceso = Celebrado, Liquidado, Terminado sin Liquidar
df_secop = df_secop.where((df_secop.Estado_Proceso == 'Celebrado') | (df_secop.Estado_Proceso == 'Liquidado') | \
                          (df_secop.Estado_Proceso == 'Terminado sin Liquidar'))

In [20]:
# Dejar moneda = No definido y Peso Colombiano
df_secop = df_secop.where((df_secop.Moneda == 'No Definido') | (df_secop.Moneda == 'Peso Colombiano'))

In [21]:
# Eliminar Anno_Cargue_SECOP = 0 (8 registros)
df_secop = df_secop.where((df_secop.Anno_Cargue_SECOP != 0))

In [22]:
# Eliminar Tipo de contrato = Consultoría, Crédito, Fiducia, Interventoría
df_secop = df_secop.where((df_secop.Tipo_Contrato != 'Consultoría') | (df_secop.Tipo_Contrato != 'Crédito') | \
                          (df_secop.Tipo_Contrato != 'Fiducia') | (df_secop.Tipo_Contrato != 'Interventoría'))

In [23]:
# Eliminar Departamentos = No definido y Colombia
df_secop = df_secop.where((df_secop.Departamento_Entidad_DANE != 'No Definido') | (df_secop.Departamento_Entidad_DANE != 'Colombia'))

In [24]:
df_secop.printSchema()

root
 |-- UID: string (nullable = false)
 |-- Anno_Cargue_SECOP: integer (nullable = true)
 |-- Anno_Firma_Contrato: integer (nullable = true)
 |-- Nivel_Entidad: string (nullable = false)
 |-- Orden_Entidad: string (nullable = false)
 |-- Tipo_Proceso: string (nullable = false)
 |-- Estado_Proceso: string (nullable = false)
 |-- Objeto_Contratar: string (nullable = false)
 |-- Detalle_Objeto_Contratar: string (nullable = false)
 |-- Tipo_Contrato: string (nullable = false)
 |-- Nombre_Grupo: string (nullable = false)
 |-- Nombre_Familia: string (nullable = false)
 |-- Nombre_Clase: string (nullable = false)
 |-- Plazo_Ejec_Contrato: integer (nullable = true)
 |-- Rango_Ejec_Contrato: string (nullable = false)
 |-- Tiempo_Adiciones_Dias: integer (nullable = true)
 |-- Tiempo_Adiciones_Meses: integer (nullable = true)
 |-- Objeto_Contrato_firmado: string (nullable = false)
 |-- Moneda: string (nullable = false)
 |-- Departamento_Entidad: string (nullable = false)
 |-- Cuantia_Proceso_In

In [25]:
# Eliminar Tipos de contrato sin interés
df_secop = df_secop.where((df_secop.Tipo_Proceso != 'Concurso de Méritos Abierto') | (df_secop.Tipo_Proceso != 'Concurso de Méritos con Lista Corta') | \
                          (df_secop.Tipo_Proceso != 'Concurso de Méritos con Lista Multiusos') | (df_secop.Tipo_Proceso != 'Concurso de diseño Arquitectónico') | \
                          (df_secop.Tipo_Proceso != 'Iniciativa Privada sin recursos públicos') | (df_secop.Tipo_Proceso != 'Lista Multiusos') | \
                          (df_secop.Tipo_Proceso != 'Llamado a presentar expresiones de interés'))

## 5 Solo contratos de obra

In [26]:
# Dejar tipo_contrato = Obra (486.949 registros)
df_secop_obra = df_secop.where((df_secop.Tipo_Contrato == 'Obra'))
#df_secop_obra.count()
#377486

In [ ]:
# Crear DataFrame en Pandas
df_secop_obra = df_secop_obra.toPandas()

In [ ]:
# Guardar dataframe en formato csv
df_secop_obra.to_csv('./df_secop_obra.csv')

# Cerrar sesión Spark

In [ ]:
spark.stop()